In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('fake_news_train.csv.zip')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
# Drop  the null value
df=df.dropna()

In [8]:
# Get the dependent features
X=df.drop('label',axis=1)

In [9]:
# Get the dependent feature
y=df['label']

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf


In [13]:
tf.__version__

'2.8.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [15]:
# vocubularty size
voc_size=5000


# Onehot Representation

In [16]:
messages=X.copy()

In [17]:
messages.reset_index(inplace=True)

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [22]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    #print(i)
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review= ' '.join(review)
    corpus.append(review)

In [23]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1073, 4698, 4419, 4005, 1336, 3345, 4119, 229, 3424, 1979],
 [2241, 4443, 1536, 1394, 505, 4247, 864],
 [2120, 3236, 2193, 190],
 [3650, 2845, 955, 57, 1878, 1046],
 [546, 505, 3738, 4439, 2180, 4227, 505, 1016, 4063, 1798],
 [1553,
  597,
  2081,
  4028,
  4414,
  3932,
  988,
  2980,
  475,
  4961,
  2455,
  4479,
  4123,
  694,
  864],
 [1654, 2426, 3493, 410, 4835, 2076, 1649, 690, 2976, 168, 4002],
 [4611, 139, 4328, 3979, 636, 3098, 3932, 1806, 2976, 168, 4002],
 [2959, 4591, 991, 4356, 1720, 881, 287, 165, 3932, 2768],
 [1607, 4856, 2302, 2548, 4222, 2605, 4886, 285],
 [2215, 493, 4389, 2237, 3554, 4402, 4356, 133, 1194, 2517, 4400],
 [57, 2446, 1336, 881, 3932, 636],
 [4985, 689, 3218, 1215, 1491, 168, 3527, 535, 1768],
 [1231, 3792, 4245, 228, 4810, 3092, 3472, 2976, 168, 4002],
 [4222, 4790, 702, 2498, 2010, 2976, 168, 4002],
 [3905, 4759, 813, 1105, 2522, 4410, 3666, 508, 4480, 4833],
 [2431, 2806, 4443],
 [2851, 2431, 3120, 4340, 3932, 7, 2088, 864],
 [2703, 1264, 1536, 4

# Embedding Represention

In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  229 3424 1979]
 [   0    0    0 ...  505 4247  864]
 [   0    0    0 ... 3236 2193  190]
 ...
 [   0    0    0 ... 2976  168 4002]
 [   0    0    0 ...  627 1163 3152]
 [   0    0    0 ...  120 2031  305]]


In [25]:
len(embedded_docs)

18285

In [27]:
##creating the model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
#model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=46)


# Model Training

In [31]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 9s 33ms/step - loss: 0.3315 - accuracy: 0.8418 - val_loss: 0.1943 - val_accuracy: 0.9176
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1418 - accuracy: 0.9448 - val_loss: 0.1962 - val_accuracy: 0.9176
Epoch 3/10
192/192 [==============================] - 11s 55ms/step - loss: 0.0984 - accuracy: 0.9626 - val_loss: 0.2518 - val_accuracy: 0.9173
Epoch 4/10
192/192 [==============================] - 11s 59ms/step - loss: 0.0655 - accuracy: 0.9783 - val_loss: 0.2356 - val_accuracy: 0.9145
Epoch 5/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0438 - accuracy: 0.9869 - val_loss: 0.3325 - val_accuracy: 0.9130
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0274 - accuracy: 0.9912 - val_loss: 0.3950 - val_accuracy: 0.9150
Epoch 7/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0180 - accuracy: 0.9940 - val_loss: 0.4248 - val_accuracy: 0.91

# Performance Metrics and Accuracy

In [40]:
y_pred=(model.predict(X_test) > 0.5).astype('int32')

In [41]:
y_pred

array([[0],
       [1],
       [0],
       ...,
       [1],
       [0],
       [1]])

In [36]:
from sklearn.metrics import confusion_matrix

In [42]:
confusion_matrix(y_test,y_pred)

array([[3142,  301],
       [ 221, 2371]], dtype=int64)